<a href="https://colab.research.google.com/github/rondinell/Intelig-ncia-Artificial/blob/main/Artigo_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install gnews
import pandas as pd
from gnews import GNews
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

# Baixa o léxico VADER (necessário para a análise de sentimento)
try:
    nltk.data.find('sentiment/vader_lexicon.zip')
except nltk.downloader.DownloadError:
    nltk.download('vader_lexicon')

# --- PARTE 1: COLETANDO NOTÍCIAS COM GNEWS ---

print("--- Iniciando a coleta de notícias para Bolsa de valores de SP ---")

# 🔍 Defina o termo de busca para a ação da Embraer
termo_busca = "Bolsa de valores de SP" # Usaremos este termo para buscar notícias

# Configurações do GNews
# Você pode ajustar o idioma, o país e o número de artigos
google_news = GNews(language='pt', country='BR', max_results=50) # Coleta até 50 notícias

# Lista para armazenar os artigos
artigos = []

try:
    # Busca por notícias relacionadas ao termo
    json_response = google_news.get_news(termo_busca)

    # Verifica se a resposta não está vazia e processa os artigos
    if json_response:
        for article in json_response:
            # Pegamos o 'title' e o 'description' (se houver) para análise de sentimento
            # A descrição pode ser o conteúdo mais relevante para o sentimento
            texto_para_analise = article.get('description', '')
            if not texto_para_analise: # Se não houver descrição, usa o título
                texto_para_analise = article.get('title', '')

            # Adiciona a data da publicação, título e o texto para análise
            # A data precisa de um tratamento se for string (normalmente vem assim)
            publicado_em = article.get('published date')
            try:
                # Tenta converter a data. O formato pode variar, então é bom ter um try-except.
                # Exemplo de formato: 'Mon, 10 Jun 2024 16:30:00 GMT'
                data_limpa = datetime.strptime(publicado_em, '%a, %d %b %Y %H:%M:%S %Z').date()
            except (ValueError, TypeError):
                data_limpa = None # Ou data.today() se preferir um default

            artigos.append([
                data_limpa,
                article.get('title'),
                texto_para_analise,
                article.get('link')
            ])
    else:
        print("Nenhuma notícia encontrada para o termo especificado.")

except Exception as e:
    print(f"Ocorreu um erro ao coletar notícias: {e}")
    print("Verifique sua conexão com a internet ou se o GNews está funcionando corretamente.")

# Criando dataframe com os artigos
df = pd.DataFrame(artigos, columns=['Data', 'Titulo', 'Conteudo para Analise', 'Link'])

# Remove linhas onde 'Conteudo para Analise' está vazio
df.replace('', pd.NA, inplace=True)
df.dropna(subset=['Conteudo para Analise'], inplace=True)

print(f"Coletados {len(df)} artigos sobre '{termo_busca}'.")
print("\n--- Primeiros artigos coletados ---")
print(df.head())

# --- PARTE 2: ANÁLISE DE SENTIMENTO ---

print("\n--- Iniciando a análise de sentimento dos artigos ---")

# Criando o analisador de sentimento VADER
sia = SentimentIntensityAnalyzer()

# Função para aplicar a análise de sentimento e extrair o score compound
def get_sentiment_compound(text):
    if isinstance(text, str) and text.strip(): # Garante que é string e não vazia
        return sia.polarity_scores(text)['compound']
    return 0.0

# Aplicando a análise de sentimento na coluna 'Conteudo para Analise'
df['compound_score'] = df['Conteudo para Analise'].apply(get_sentiment_compound)

# Função para classificar o sentimento com base no 'compound_score'
def classify_sentiment(score):
    if score >= 0.05:
        return "Positivo"
    elif score <= -0.05:
        return "Negativo"
    else:
        return "Neutro"

# Criando uma nova coluna 'sentimento' com a classificação
df['sentimento'] = df['compound_score'].apply(classify_sentiment)

print("\n--- Análise de Sentimento Concluída ---")
print("\n--- DataFrame com Sentimento ---")
print(df.head())

# --- PARTE 3: VISUALIZAÇÃO DOS RESULTADOS (Opcional) ---

print("\n--- Resumo da Análise de Sentimento ---")
print(df['sentimento'].value_counts())

# Sentimento por dia (se as datas foram coletadas corretamente)
if 'Data' in df.columns and not df['Data'].isnull().all():
    print("\n--- Sentimento por Dia ---")
    sentimento_por_dia = df.groupby('Data')['sentimento'].value_counts().unstack(fill_value=0)
    print(sentimento_por_dia)
else:
    print("\nNão foi possível agrupar por dia, pois a coluna 'Data' está ausente ou vazia.")

--- Iniciando a coleta de notícias para Bolsa de valores de SP ---
Coletados 50 artigos sobre 'Bolsa de valores de SP'.

--- Primeiros artigos coletados ---
         Data                                             Titulo  \
0  2025-06-16  Dólar recua e vai a R$ 5,49, em semana de Copo...   
1  2025-06-12  Bolsa oscila e dólar fica estável após divulga...   
2  2025-04-07  Bolsa brasileira tem 3ª queda seguida em meio ...   
3  2025-05-20  Bolsa volta a bater recorde e supera os 140 mi...   
4  2025-03-14  Deputados acompanham leilão de rodovias estadu...   

                               Conteudo para Analise  Link  
0  Dólar recua e vai a R$ 5,49, em semana de Copo...  None  
1  Bolsa oscila e dólar fica estável após divulga...  None  
2  Bolsa brasileira tem 3ª queda seguida em meio ...  None  
3  Bolsa volta a bater recorde e supera os 140 mi...  None  
4  Deputados acompanham leilão de rodovias estadu...  None  

--- Iniciando a análise de sentimento dos artigos ---

--- Análise 